In [174]:
import os
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
import json
# import tesseract
import pytesseract
from typing import *
from box import ConfigBox
from pathlib import Path
import re, sys, os
from functools import partial



import io
import os
import base64
import numpy as np
from PIL import Image
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage, AIMessage


import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

from dotenv import load_dotenv

from dataclasses import dataclass

In [39]:
load_dotenv()

True

In [26]:
from backend.src.utils.common import *
from backend.logger import logger
from backend.src.constants import *


In [22]:
os.getcwd()
os.chdir("../../")

In [23]:
read_yaml(CONFIG_FILE_PATH)

2025-02-07 16:17:27,576 - root - INFO - Yaml read successfully from config.yaml


ConfigBox({'data_dir': {'raw_data': 'data/raw_pdfs', 'reports': 'data/reports'}})

In [24]:
%load_ext watermark
%watermark -a "Rajesh goldy" -vmp langchain,unstructured,openai,pydantic,pytesseract

Author: Rajesh goldy

Python implementation: CPython
Python version       : 3.9.21
IPython version      : 8.18.1

langchain   : 0.3.17
unstructured: 0.16.11
openai      : 1.61.1
pydantic    : 2.10.6
pytesseract : 0.3.13

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [81]:
class Extractor:
    def __init__(self):
        pass
    
    def get_metadata(raw_pdf_element: List) -> List:
        """To get the metadata of elements

        Args:
            raw_pdf_element (List): list of elements in pdf

        Returns:
            List: containing meta data
        """
        metadata = []
        for element in raw_pdf_element:
            metadata.append(element.metadata.to_dict())
        
        return metadata
    
    def get_year(file: str) -> str:
        """To get the year from file name
        Args:
            file: name of with extension"""
        
        year = re.search(r"\d{4}", file).group()    
        if year:
            return str(year)
        else:
            try:
                file_name = os.path.split(file)[-1].split(".")[0]
                return file_name
            except Exception as e:
                logger.error(f"Error in getting year from file: {file} with error: {e}")
                return "1.0"
        
    def extract_data(self, pdf_file, **kwargs):
        raw_pdf_elements = partition_pdf(filename=pdf_file, **kwargs)
        return raw_pdf_elements
    
    def generate_unique_id(self, data: Iterator)-> List[str]:
        """To generate unique id for each element"""
        
        unique_id = []
        for _ in data:
            unique_id.append(str(uuid.uuid4()))
        
        return unique_id
        

In [176]:
from unstructured.documents.elements import CompositeElement, Table, Image, ListItem  # Import ListItem


@dataclass
class ImageExtractorConfig:
    raw: str # raw pdf path
    reports: str # processed reports path
    
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH, PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
    def get_image_extractor_params(self) -> ImageExtractorConfig:
        params = ImageExtractorConfig(
            raw=self.config.data_dir.raw,
            reports=self.config.data_dir.reports)
        return params
    
    
class ImageExtractor(Extractor):
    def __init__(self, config):
        super(ImageExtractor, self).__init__()
        self.config = config
    
    def process_pdf(self, pdf_file_path: str, **kwargs):
        """To extract the images from pdf file using unstructured library

        Args:
            pdf_file_path (str): full path of pdf file

        Returns:
            _type_: _description_
        """
        report_year = Extractor.get_year(pdf_file_path) # get the year from file name and set it as directory name
        pdf_dir = Path(pdf_file_path).resolve().parent # get the directory of pdf file   backend/data/raw_pdfs -> backend/data
        
        image_dir = os.path.join(self.config.reports, report_year) 
        print("image Dir: ", image_dir)
        # to store the images extracted from pdf
        create_directory(image_dir, is_extension_present=False) # create directory with year name
        print("kwargs before", kwargs)
        if kwargs.get("extract_image_block_output_dir", None) == None:
            kwargs["extract_image_block_output_dir"] = image_dir
        print("kwargs after", kwargs)
        raw_pdf_elements = self.extract_data(pdf_file_path, **kwargs)
        
        images = [str(e) for e in raw_pdf_elements if isinstance(e, Image)]
        text = [str(e) for e in raw_pdf_elements if isinstance(e, CompositeElement)]
        
        return ConfigBox({
            "images": raw_pdf_elements,
            "text": text
            })
    

                

In [169]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
image_extractor_config = config_manager.get_image_extractor_params()
image_extractor = ImageExtractor(image_extractor_config)

2025-02-07 23:42:24,129 - root - INFO - Yaml read successfully from config.yaml
2025-02-07 23:42:24,132 - root - ERROR - FileNotFoundError: params.yaml


In [170]:
image_extractor_config

ImageExtractorConfig(raw='backend/data/raw_pdfs', reports='backend/data/reports')

In [171]:
os.getcwd()

'/Users/goldyrana/mess/deep_learning/projects/rag'

In [172]:
os.listdir(image_extractor_config.raw)

['2023', '2023_removed.pdf', '2024_removed.pdf']

In [ ]:
images, text = images

In [173]:
images, text = image_extractor.process_pdf(pdf_file_path="backend/data/raw_pdfs/2023_removed.pdf",
                                    strategy="hi_res", # 
                                    split_pdf_page=True,  # to process each page seprately
                                    split_pdf_allow_failed=True, # continue processing even if some pages fail
                                    extract_images_in_pdf=True,
                                    infer_table_structure=True,
                                    chunking_strategy="by_title",
                                    extract_image_block_types = ["Image" , "Table"],
                                    max_characters=4000,
                                    new_after_n_chars=3800,
                                    combine_text_under_n_chars=2000
                                    )

2025-02-07 23:42:27,095 - pdfminer.psparser - DEBUG - seek: 0
2025-02-07 23:42:27,097 - pdfminer.pdfdocument - DEBUG - find_xref: b''


2025-02-07 23:42:27,097 - pdfminer.pdfdocument - DEBUG - find_xref: b'%%EOF'
2025-02-07 23:42:27,109 - pdfminer.pdfdocument - DEBUG - find_xref: b'10081360'
2025-02-07 23:42:27,161 - pdfminer.pdfdocument - DEBUG - find_xref: b'startxref'
2025-02-07 23:42:27,162 - pdfminer.pdfdocument - DEBUG - xref found: pos=b'10081360'
2025-02-07 23:42:27,162 - pdfminer.psparser - DEBUG - seek: 10081360
2025-02-07 23:42:27,163 - pdfminer.psparser - DEBUG - nexttoken: (10081360, /b'xref')
2025-02-07 23:42:27,163 - pdfminer.pdfdocument - DEBUG - read_xref_from: start=10081360, token=/b'xref'
2025-02-07 23:42:27,164 - pdfminer.psparser - DEBUG - nextline: 10081364, b'\n'
2025-02-07 23:42:27,164 - pdfminer.psparser - DEBUG - nextline: 10081365, b'0 62\n'
2025-02-07 23:42:27,165 - pdfminer.psparser - DEBUG - nextline: 10081370, b'0000000000 65535 f\r\n'
2025-02-07 23:42:27,166 - pdfminer.psparser - DEBUG - nextline: 10081390, b'0010081239 00000 n\r\n'
2025-02-07 23:42:27,167 - pdfminer.psparser - DEBUG - 

image Dir:  backend/data/reports/2023
kwargs before {'strategy': 'hi_res', 'split_pdf_page': True, 'split_pdf_allow_failed': True, 'extract_images_in_pdf': True, 'infer_table_structure': True, 'chunking_strategy': 'by_title', 'extract_image_block_types': ['Image', 'Table'], 'max_characters': 4000, 'new_after_n_chars': 3800, 'combine_text_under_n_chars': 2000}
kwargs after {'strategy': 'hi_res', 'split_pdf_page': True, 'split_pdf_allow_failed': True, 'extract_images_in_pdf': True, 'infer_table_structure': True, 'chunking_strategy': 'by_title', 'extract_image_block_types': ['Image', 'Table'], 'max_characters': 4000, 'new_after_n_chars': 3800, 'combine_text_under_n_chars': 2000, 'extract_image_block_output_dir': 'backend/data/reports/2023'}


2025-02-07 23:42:27,284 - pdfminer.psparser - DEBUG - nexttoken: (2182673, /b'>>')
2025-02-07 23:42:27,284 - pdfminer.psparser - DEBUG - end_type: pos=2182637, type='d', objs=[/'Filter', /'FlateDecode', /'Length', 647]
2025-02-07 23:42:27,285 - pdfminer.psparser - DEBUG - nexttoken: (2182676, /b'stream')
2025-02-07 23:42:27,285 - pdfminer.psparser - DEBUG - do_keyword: pos=2182676, token=/b'stream', stack=[(2182637, {'Filter': /'FlateDecode', 'Length': 647})]
2025-02-07 23:42:27,285 - pdfminer.psparser - DEBUG - seek: 2182676
2025-02-07 23:42:27,286 - pdfminer.psparser - DEBUG - nextline: 2182676, b'stream\n'
2025-02-07 23:42:27,286 - pdfminer.psparser - DEBUG - seek: 2183330
2025-02-07 23:42:27,286 - pdfminer.psparser - DEBUG - nextline: 2183330, b'\n'
2025-02-07 23:42:27,286 - pdfminer.psparser - DEBUG - nextline: 2183331, b'endstream\n'
2025-02-07 23:42:27,287 - pdfminer.psparser - DEBUG - seek: 2183331
2025-02-07 23:42:27,287 - pdfminer.pdfparser - DEBUG - Stream: pos=2182683, objl

TypeError: isinstance() arg 2 must be a type or tuple of types

In [160]:
for i in images.images:
    print(type(i))
    print(str(i))

<class 'unstructured.documents.elements.CompositeElement'>
Cautionary Statement

This presentation provides management's current operational plan for ConocoPhillips over roughly the next decade, for the assets currently in our portfolio, and is subject to multiple assumptions, including, unless otherwise specifically noted: * anoil price of $60/BBL West Texas Intermediate in 2022 dollars, escalating at 2.25% annually;

* anoil price of $65/BBL Brent in 2022 dollars, escalating at 2.25% annually;

* agas price of $3.75/MMBTU Henry Hub in 2022 dollars, escalating at 2.25%

annually;

an international gas price of $8/MMBTU Title Transfer Facility & Japan Korea Marker in 2022 dollars, escalating at 2.25% annually;

* cost and capital escalation in line with price escalation; planning case at $60/BBL WTI assumes capital de-escalation from levels

observed in 2022;

* all production compound annual growth rates (CAGR) are calculated for the 10-year period 2023 — 2032;

inclusion of carbon ta

In [34]:
@dataclass
class ImageSummarizerConfig:
    model: str #name of model
    
    
def get_image_summarizer_params(self) -> ImageSummarizerConfig:
    params = ImageSummarizerConfig(
        model=self.config.model.chat_model)
    return params
    
class ImageSummarizer(Extractor):
    def __init__(self, config, model: object):
        super().__init__()
        self.config = config
        self.model = model
            
    def encode_image(self, image_path):
        """Encode image to base64
        Args:
            image_path: path to image
        Returns:
            str: base64 encoded image"""
        try:
            with open(image_path, "rb") as image:
                return base64.b64encode(image.read()).decode("utf-8")
        except Exception as e:
            print(f"Error: {e}")
            raise e
        
    def image_summarize(self, img_base64, prompt) -> str:
        """Summarize image using LLM model API
        Args:
            img_base64: base64 encoded image
            prompt: prompt to summarize image
        Returns:
            str: summary of image"""
        
        msg = self.model.invoke(
            [
                HumanMessage(
                    content=[
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{img_base64}"
                            },
                        },
                    ]
                )
            ]
        )
        return msg.content
    
    def store_summaries_to_vb(self, encoded_images, image_summaries, id_key: str, retriever: object) -> None:
        """Store images(Encoded images) and summaries to vectorbase

        Args:
            encoded_images (_type_): base64 encoded images
            image_summaries (_type_): Summaries of images generated by model
            id_key (str): Unique ID key for file processing
            retriever (object): Vectorbase retriever object
        Returns:
            None
        """
        image_ids = Extractor.generate_unique_id(image_summaries)
        image_summaries = [Document(page_content=summary.content, metadata={id_key: image_ids[index]})
                         for index, summary in enumerate(image_summaries)]
        
        retriever.vectorstore.add_documents(image_summaries)
        retriever.docstore.mset(list(zip(image_ids, encoded_images)))
        print("Stored images and summaries to vectorbase")


# Multivector storage


In [51]:
from langchain_chroma import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever


In [53]:
vectorstore = Chroma("test", OpenAIEmbeddings(), persist_directory="test_vectorstore")   
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

2025-02-07 22:43:36,059 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-07 22:43:36,065 - chromadb.config - DEBUG - Starting component System
2025-02-07 22:43:36,065 - chromadb.config - DEBUG - Starting component Posthog
2025-02-07 22:43:36,066 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-07 22:43:36,066 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-07 22:43:36,110 - chromadb.config - DEBUG - Starting component SimpleQuotaEnforcer
2025-02-07 22:43:36,111 - chromadb.config - DEBUG - Starting component SimpleRateLimitEnforcer
2025-02-07 22:43:36,111 - chromadb.config - DEBUG - Starting component LocalSegmentManager
2025-02-07 22:43:36,112 - chromadb.config - DEBUG - Starting component LocalExecutor
2025-02-07 22:43:36,112 - chromadb.config - DEBUG - Starting component SegmentAPI
